In [175]:
import pandas as pd
import numpy as np

### Preprocessing for all possible TABLES

In [176]:
df = pd.read_csv("./UH_2023/UH_2023_TRAIN.txt", sep = "|")

In [177]:
df = df.astype({"CAMPAÑA": str, "ID_FINCA": str, "ID_ZONA": str, "ID_ESTACION": str, "VARIEDAD": str, "MODO": str, "TIPO": str, "COLOR": str})

In [178]:
def fun(x, mode): 
    altitud = str(x).split('-')
    if altitud[0] == "nan":
        if mode == "nan_1":
            return -1
        return np.nan
    if mode == "nan_1":
            return x
    if mode == "min":
        return int(altitud[0])
    if mode == "diff":
        if len(altitud) == 1:
            return 0
        else:
            return int(altitud[1])-int(altitud[0])

df["ALTITUD_MIN"] = df["ALTITUD"].apply(fun, args=("min",))
df["ALTITUD_DIF"] = df["ALTITUD"].apply(fun, args=("diff",))
df["ALTITUD"] = df["ALTITUD"].apply(fun, args=("nan_1",))

In [179]:
df_estacion_gb_mean = df.groupby(by="ID_ESTACION").mean()
df_estacion_gb_mode = df.groupby(by="ID_ESTACION").agg(pd.Series.mode)

C:\Users\xavid\anaconda3\lib\site-packages\pandas\core\algorithms.py:968: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  warn(f"Unable to sort modes: {err}")
C:\Users\xavid\AppData\Local\Temp\ipykernel_516\3219566741.py:2: FutureWarning: ['SUPERFICIE', 'PRODUCCION'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df_estacion_gb_mode = df.groupby(by="ID_ESTACION").agg(pd.Series.mode)


In [180]:
df_estacion_gb_mean.reset_index(inplace=True)
df_estacion_gb_mode.reset_index(inplace=True)

In [181]:
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "0", "ALTITUD"] = 390
df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == "4", "ALTITUD"] = 520

In [182]:
for i in df["ID_ESTACION"].unique():
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_MIN"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_MIN"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD_DIF"] = df_estacion_gb_mean.loc[df_estacion_gb_mean["ID_ESTACION"] == i,"ALTITUD_DIF"].values[0]
    df.loc[(df["ID_ESTACION"] == i) & (df["ALTITUD"] == -1),"ALTITUD"] = df_estacion_gb_mode.loc[df_estacion_gb_mode["ID_ESTACION"] == i,"ALTITUD"].values[0]

In [183]:
df.loc[df["MODO"]=="1", "MODO"]  = 0
df.loc[df["MODO"]=="2", "MODO"]  = 1

In [184]:
df = df.drop(index=df[df["CAMPAÑA"] == "22"].index)

### 1 - Table without SUPERFICIE

In [185]:
df_temporal = df.drop(axis = 1, columns = ["SUPERFICIE"])

In [186]:
import category_encoders as ce

encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])
df_encoded = encoder.fit_transform(df_temporal)
encoder = ce.OrdinalEncoder(cols = ["CAMPAÑA"])
df_encoded = encoder.fit_transform(df_temporal)

In [169]:
df_temporal.to_csv("D_T_1.csv")
df_encoded.to_csv("D_T_1_encoded.csv")

### 2 - Table without CAMPAÑA and SUPERFICIE

In [187]:
df_temporal = df.drop(axis = 1, columns = ["CAMPAÑA", "SUPERFICIE"])

In [188]:
encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])

df_encoded = encoder.fit_transform(df_temporal)

In [172]:
df_temporal.to_csv("D_T_2.csv")
df_encoded.to_csv("D_T_encoded_2.csv")

### 3 - Table without CAMPAÑA

In [189]:
df_temporal = df.drop(axis = 1, columns = ["CAMPAÑA"])

In [190]:
encoder = ce.OneHotEncoder(cols=["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "ALTITUD"])

df_encoded = encoder.fit_transform(df_temporal)

In [197]:
train = df_encoded[(df_encoded["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0)]
to_predict = df_encoded[~((df_encoded["PRODUCCION"] != 0) & (df_encoded["SUPERFICIE"] != 0))]

In [198]:
X = train.drop(axis = 1, columns = ["SUPERFICIE"])
y = train["SUPERFICIE"]
to_predict = to_predict.drop(axis = 1, columns = ["SUPERFICIE"])

In [199]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor


model = {"model_name": "SVC", "model": MLPRegressor(), "params": {'hidden_layer_sizes':[(100),(100,100,100)],
                                                                  'learning_rate':["constant", "adaptive"],}}

grid_search = GridSearchCV(model["model"], model["params"], scoring="r2")
grid_search.fit(X, y)

GridSearchCV(estimator=MLPRegressor(),
             param_grid={'hidden_layer_sizes': [100, (100, 100, 100)],
                         'learning_rate': ['constant', 'adaptive']},
             scoring='r2')

In [200]:
print(grid_search.best_params_)
df = pd.DataFrame(grid_search.cv_results_)
df

{'hidden_layer_sizes': (100, 100, 100), 'learning_rate': 'constant'}


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_hidden_layer_sizes,param_learning_rate,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,9.912110,3.713639,0.055784,0.005744,100,constant,"{'hidden_layer_sizes': 100, 'learning_rate': '...",-0.232126,-0.227700,0.625740,-0.312982,0.375223,0.045631,0.380951,2
1,6.092636,1.570305,0.055397,0.005204,100,adaptive,"{'hidden_layer_sizes': 100, 'learning_rate': '...",-0.094918,-5.629649,0.538165,-0.260796,-12.144521,-3.518344,4.850882,4
2,5.612677,0.905608,0.051236,0.005237,"(100, 100, 100)",constant,"{'hidden_layer_sizes': (100, 100, 100), 'learn...",-0.016260,0.354650,0.537014,0.445591,0.377986,0.339796,0.188967,1
3,4.739089,1.153451,0.046161,0.002064,"(100, 100, 100)",adaptive,"{'hidden_layer_sizes': (100, 100, 100), 'learn...",0.415456,0.360087,0.091130,-11.162598,-1.154376,-2.290060,4.472692,3


In [204]:
modelMLPR = MLPRegressor(hidden_layer_sizes=(100), learning_rate="adaptive")
modelMLPR.fit(X, y)
predicted = modelMLPR.predict(to_predict)
final_df = to_predict
final_df["SUPERFICIE"] = predicted

In [212]:
sum(predicted<0)

39

In [215]:
final_df = pd.concat([final_df, train])

In [219]:
final_df

,ID_FINCA_1,ID_FINCA_2,ID_FINCA_3,ID_FINCA_4,ID_FINCA_5,ID_FINCA_6,ID_FINCA_7,ID_FINCA_8,ID_FINCA_9,ID_FINCA_10,...,VARIEDAD_23,VARIEDAD_24,VARIEDAD_25,MODO,TIPO,COLOR,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF,SUPERFICIE
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,22215.0,660.000000,0.00000,4.515938
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,22215.0,660.000000,0.00000,4.515956
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,20978.0,520.000000,0.00000,4.092378
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,1,0,1,40722.0,520.000000,0.00000,6.868805
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,14126.0,654.501718,9.19244,3.540188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8521,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,28160.1,700.000000,0.00000,3.680000
8522,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,41310.0,700.000000,0.00000,4.250000
8523,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,45420.0,700.000000,0.00000,4.160000
8524,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,56140.0,700.000000,0.00000,4.750000


In [224]:
final_df.iloc[1143:1144,:]

,ID_FINCA_1,ID_FINCA_2,ID_FINCA_3,ID_FINCA_4,ID_FINCA_5,ID_FINCA_6,ID_FINCA_7,ID_FINCA_8,ID_FINCA_9,ID_FINCA_10,...,VARIEDAD_23,VARIEDAD_24,VARIEDAD_25,MODO,TIPO,COLOR,PRODUCCION,ALTITUD_MIN,ALTITUD_DIF,SUPERFICIE
1143,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,30270.6,700.0,0.0,5.670536


In [ ]:
final_df.to_csv("D_T_3.csv")
final_df.to_csv("D_T_encoded_3.csv")